In [1]:
import pandas as pd

df = pd.read_csv("Agriculture_price_dataset.csv")

In [2]:

# Keep only rows where state = Maharashtra
df = df[df['STATE'] == 'Maharashtra']

df.to_csv("filtered.csv", index=False)

In [18]:
unique_pairs = df[['District Name','Market Name']].drop_duplicates()

for i in unique_pairs.iterrows():
    if "Market" in i[1][1] or "Bazar" in i[1][1]:
        print(i[1][1])

Chandur Bazar
Amrawati(Frui & Veg. Market)
Sangli(Phale, Bhajipura Market)
Varud(Rajura Bazar)
Laxmi Sopan Agriculture Produce Marketing Co Ltd
Shree Salasar Krushi Bazar
Shree Rameshwar Krushi Market
Shree Sairaj Krushi Market
Janata Agri Market (DLS Agro Infrastructure Pvt Lt
Sant Namdev Krushi Bazar,
Premium Krushi Utpanna Bazar


C:\Users\Atharv\AppData\Local\Temp\ipykernel_21468\2051475019.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if "Market" in i[1][1] or "Bazar" in i[1][1]:
C:\Users\Atharv\AppData\Local\Temp\ipykernel_21468\2051475019.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(i[1][1])


In [35]:
df["District Name"] = df["District Name"].str.lower()
df["Market Name"] = df["Market Name"].str.lower()

In [53]:
df2 =  pd.read_csv("cities.csv")
df2["city"] = df2["city"].str.lower()
df2["district"] = df2["district"].str.lower()
df2 = df2.drop_duplicates(subset=["district"])
df2["district"] = df2["district"].replace("amravati", "amarawati")
merged = df.merge(df2, left_on="District Name", right_on="district", how="left")


print(merged)

             STATE District Name                             Market Name  \
0      Maharashtra        nashik                       lasalgaon(niphad)   
1      Maharashtra        satara                                   patan   
2      Maharashtra        nashik                      lasalgaon(vinchur)   
3      Maharashtra    ahmednagar                                pathardi   
4      Maharashtra         dhule  pratap nana mahale khajgi bajar samiti   
...            ...           ...                                     ...   
51811  Maharashtra          pune                            khed(chakan)   
51812  Maharashtra    chandrapur                     chandrapur(ganjwad)   
51813  Maharashtra          pune                           pune(khadiki)   
51814  Maharashtra       jalgaon                                bhusaval   
51815  Maharashtra        nashik                              pimpalgaon   

      Commodity           Variety  Grade  Min_Price  Max_Price  Modal_Price  \
0       

In [55]:
merged = merged.dropna(subset=["city"])
merged.to_csv("joined.csv", index=False)

In [56]:
test = merged.copy()
test["Price Date"] = pd.to_datetime(test["Price Date"], errors="coerce")
test.to_csv("date.csv",  index = True)

In [30]:
for i, roes in unique_pairs.iterrows():
    print(i, roes)

0 District Name               nashik
Market Name      Lasalgaon(Niphad)
lat                       9.904997
lon                       78.10483
Name: 0, dtype: object
1 District Name       satara
Market Name          Patan
lat              18.906836
lon              75.674158
Name: 1, dtype: object
19 District Name                nashik
Market Name      Lasalgaon(Vinchur)
lat                        9.904997
lon                        78.10483
Name: 19, dtype: object
39 District Name    ahmednagar
Market Name        Pathardi
lat               18.906836
lon               75.674158
Name: 39, dtype: object
41 District Name                                     dhule
Market Name      Pratap Nana Mahale Khajgi Bajar Samiti
lat                                           18.906836
lon                                           75.674158
Name: 41, dtype: object
42 District Name       satara
Market Name        Phaltan
lat              18.906836
lon              75.674158
Name: 42, dtype: object
44 Dis

In [57]:
sdf = pd.read_csv("semifinal.csv")
A = sdf.head()

In [59]:
A.to_csv('test.csv')

In [60]:
sem = pd.read_csv("semifinal.csv")
test = sem.head()
test.to_csv("test.csv")

In [10]:
models= dict()

In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor # Keeping import, but logic removed (for code cleanliness, can be removed entirely)
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, make_scorer
from sklearn.preprocessing import StandardScaler
import pickle 

# --- Configuration ---
INPUT_FILE = 'rice_market_lagged_features.csv' 
TARGET_COLUMN = 'Modal_Price'
TEST_SPLIT_RATIO = 0.2 
N_SPLITS = 5 
COMMODITY_NAME = 'Rice'

# --- GPU Configuration ---
# Set the device for GPU acceleration. Use 'cuda' for NVIDIA GPUs.
# Note: This requires the XGBoost library to be compiled with GPU support
# and the presence of NVIDIA drivers/CUDA toolkit on the execution environment.
GPU_ENABLED_PARAMS = {
    'tree_method': 'hist', # Use 'hist' or 'gpu_hist' for modern XGBoost versions
    'device': 'cuda'
}


def load_and_prepare_data(file_path):
    """Loads the feature-engineered data and separates features (X) and target (y)."""
    try:
        # Index columns are Market Name and Price Date
        df = pd.read_csv(file_path, index_col=[0, 1], parse_dates=True)
        print(f"Loaded data successfully. Total records: {len(df)}")
    except FileNotFoundError:
        print(f"Error: Input file '{file_path}' not found. Ensure data_preparation.py was run successfully.")
        return None, None, None

    # Separate target (y) and features (X)
    y = df[TARGET_COLUMN]
    
    # Features (X) include all lagged features and all current-week weather data
    X = df.drop(columns=['Min_Price', 'Max_Price', TARGET_COLUMN], errors='ignore')
    
    return df, X, y

def split_and_scale_data(X, y):
    """Splits the data temporally and scales the features."""
    
    # 1. Temporal Split (Train/Test)
    split_index = int(len(X) * (1 - TEST_SPLIT_RATIO))
    
    X_train, X_test = X.iloc[:split_index], X.iloc[split_index:]
    y_train, y_test = y.iloc[:split_index], y.iloc[split_index:]

    print(f"\nTraining set size: {len(X_train)} weeks")
    print(f"Test set size: {len(X_test)} weeks")
    
    # 2. Scaling 
    # Use StandardScaler (Z-score normalization) to prevent magnitude-based bias
    scaler = StandardScaler()
    # Fit scaler only on training data to prevent data leakage
    X_train_scaled = scaler.fit_transform(X_train)
    # Apply the same scaler to test data
    X_test_scaled = scaler.transform(X_test)
    
    # Convert scaled NumPy arrays back to DataFrames for feature names
    X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
    X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.columns)
    
    return X_train_scaled_df, X_test_scaled_df, y_train, y_test, scaler

def evaluate_models(X_train, X_test, y_train, y_test):
    """
    Trains and evaluates the XGBoost baseline model for comparison with the tuned version.
    Now includes GPU acceleration parameters.
    """
    
    # Combine standard params with GPU params
    xgb_params = {
        'objective': 'reg:squarederror', 
        'n_estimators': 200, 
        'learning_rate': 0.05, 
        'max_depth': 6, 
        'random_state': 42, 
        'n_jobs': -1,
        'min_child_weight': 1.5,
        **GPU_ENABLED_PARAMS # <-- Added GPU parameters
    }

    models = {
        "XGBoost": XGBRegressor(**xgb_params)
    }

    results = {}
    
    for name, model in models.items():
        print(f"\nTraining Baseline {name} Regressor (using GPU if available)...")
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        mae = mean_absolute_error(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)
        
        results[name] = {'MAE': mae, 'RMSE': rmse, 'R2': r2, 'model': model}

        print(f"--- {name} Performance (Baseline) ---")
        print(f"MAE: {mae:.2f} INR")
        print(f"RMSE: {rmse:.2f} INR")
        print(f"R-squared: {r2:.4f}")
        
    return results

def tune_model_with_tscv(X_train, y_train):
    """
    Performs Randomized Hyperparameter Search for XGBoost using TimeSeriesSplit (Step 3).
    Now includes GPU acceleration parameters.
    """
    print("\n" + "="*50)
    print("STEP 3: HYPERPARAMETER TUNING (XGBoost + TimeSeriesSplit)")
    print("="*50)

    # Define the parameter grid for Randomized Search
    param_grid = {
        'n_estimators': [200, 300, 500],
        'max_depth': [3, 5, 7],
        'learning_rate': [0.1, 0.05, 0.01, 0.005],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
    }

    # Initialize XGBoost model with GPU parameters
    xgb = XGBRegressor(
        objective='reg:squarederror', 
        random_state=42, 
        n_jobs=-1,
        **GPU_ENABLED_PARAMS # <-- Added GPU parameters
    )

    # TimeSeriesSplit is crucial here to prevent data leakage
    tscv = TimeSeriesSplit(n_splits=N_SPLITS)
    
    # Use R2 score as the metric for optimization
    scorer = make_scorer(r2_score)

    # Setup Randomized Search
    random_search = RandomizedSearchCV(
        estimator=xgb,
        param_distributions=param_grid,
        scoring=scorer,
        cv=tscv,
        n_iter=90, # Number of parameter settings that are sampled
        verbose=1,
        random_state=42,
        n_jobs=-1
    )

    print("Starting Randomized Search (10 iterations)... This may take a few moments.")
    # Fit the search on the training data
    random_search.fit(X_train, y_train)
    
    print("\n--- Hyperparameter Tuning Results (Cross-Validation Score) ---")
    print(f"Best R2 Score found during CV: {random_search.best_score_:.4f}")
    print("Best Parameters:")
    for k, v in random_search.best_params_.items():
        print(f"  {k}: {v}")
        
    return random_search.best_estimator_

def get_feature_importances(model, feature_names):
    """Extracts and formats feature importances."""
    if hasattr(model, 'feature_importances_'):
        importance_scores = model.feature_importances_
        feature_importances = pd.Series(importance_scores, index=feature_names).sort_values(ascending=False)
        return feature_importances
    return None

def final_evaluation(model, X_test, y_test, model_name="Tuned XGBoost"):
    """Evaluates the final best model on the hold-out test set."""
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    
    print(f"\n--- {model_name} Performance on FINAL Test Set ---")
    print(f"MAE: {mae:.2f} INR (Average absolute price prediction error)")
    print(f"RMSE: {rmse:.2f} INR")
    print(f"R-squared: {r2:.4f} (Variance explained)")
    
    return r2

def save_model_artifacts(model, scaler, model_filename="tuned_xgb_model.pkl", scaler_filename="scaler.pkl"):
    """
    Saves the final trained model and the scaler object using pickle.
    """
    # Save the Model
    try:
        with open(model_filename, 'wb') as file:
            pickle.dump(model, file)
        print(f"\nModel successfully saved to: {model_filename}")
    except Exception as e:
        print(f"Error saving model: {e}")

    # Save the Scaler
    # NOTE: The scaler MUST be saved and loaded with the model. Any new data 
    # must be scaled using this exact scaler before prediction.
    try:
        with open(scaler_filename, 'wb') as file:
            pickle.dump(scaler, file)
        print(f"Scaler successfully saved to: {scaler_filename}")
    except Exception as e:
        print(f"Error saving scaler: {e}")
        

if __name__ == "__main__":
    df_data, X, y = load_and_prepare_data(INPUT_FILE)
    
    if X is not None:
        # Step 1: Split and Scale Data
        X_train, X_test, y_train, y_test, scaler = split_and_scale_data(X, y)
        
        # Step 2: Baseline Model Evaluation (XGBoost only)
        baseline_results = evaluate_models(X_train, X_test, y_train, y_test)
        
        # Step 3: Hyperparameter Tuning
        best_xgb_model = tune_model_with_tscv(X_train, y_train)
        
        # Step 4: Final Evaluation of the Tuned Model
        print("\n" + "="*50)
        print("FINAL MODEL EVALUATION")
        print("="*50)
        tuned_r2 = final_evaluation(best_xgb_model, X_test, y_test)
        
        # Comparison with Baseline XGBoost
        baseline_xgb_r2 = baseline_results['XGBoost']['R2']
        r2_gain = tuned_r2 - baseline_xgb_r2
        print(f"\nBaseline XGBoost R2: {baseline_xgb_r2:.4f}")
        print(f"Tuning improved R2 by: {r2_gain:.4f}")
        
        # Feature Importance for the best model
        feature_importances = get_feature_importances(best_xgb_model, X_train.columns)
        print(f"\n--- Top 10 Feature Importances (from Tuned XGBoost) ---")
        print(feature_importances.head(10))
        
        # -----------------------------------------------------
        # NEW STEP: Save the Model and Scaler for Deployment
        # -----------------------------------------------------
        print("\n" + "="*50)
        print("SAVING MODEL ARTIFACTS")
        print("="*50)
        model_name = COMMODITY_NAME + ".pkl"
        save_model_artifacts(best_xgb_model, scaler, model_filename=model_name)
        
        print("\n--- NEXT STEPS ---")
        print("The model and its scaler are saved. You can now load these files in a separate script for production use.")
        print("1. Deployment: Implement the tuned XGBoost model to make predictions on future data.")
        print("2. Further Improvement: Try increasing the `LAG_WINDOW` in data_preparation.py or optimizing the Random Forest model.")

C:\Users\Atharv\AppData\Local\Temp\ipykernel_34968\1832342684.py:31: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(file_path, index_col=[0, 1], parse_dates=True)


Loaded data successfully. Total records: 36

Training set size: 28 weeks
Test set size: 8 weeks

Training Baseline XGBoost Regressor (using GPU if available)...
--- XGBoost Performance (Baseline) ---
MAE: 177.85 INR
RMSE: 238.82 INR
R-squared: 0.3243

STEP 3: HYPERPARAMETER TUNING (XGBoost + TimeSeriesSplit)
Starting Randomized Search (10 iterations)... This may take a few moments.
Fitting 5 folds for each of 90 candidates, totalling 450 fits

--- Hyperparameter Tuning Results (Cross-Validation Score) ---
Best R2 Score found during CV: -1.2498
Best Parameters:
  subsample: 0.6
  n_estimators: 300
  max_depth: 3
  learning_rate: 0.005
  colsample_bytree: 0.6

FINAL MODEL EVALUATION

--- Tuned XGBoost Performance on FINAL Test Set ---
MAE: 255.04 INR (Average absolute price prediction error)
RMSE: 293.44 INR
R-squared: -0.0202 (Variance explained)

Baseline XGBoost R2: 0.3243
Tuning improved R2 by: -0.3444

--- Top 10 Feature Importances (from Tuned XGBoost) ---
Modal_Price_Lag3         